# 1. Data Retrieval

**TODO:** Copy from 

One for manual inspection (csv) and one conservative (pkl)

Loads necessary libraries for the EDA:

In [1]:
import os
import subprocess
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv

Global configs:

In [2]:
# Path to root directory of the repo.
root_dir_ = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"],
    text=True,
)
ROOT_DIR = root_dir_.strip()

# Path to data directory.
DATA_DIR = os.path.join(ROOT_DIR, "data")

# Paths to which dataframes will be saved
DF_PKL_PATH = os.path.join(DATA_DIR, "df.pkl")
DF_CSV_PATH = os.path.join(DATA_DIR, "df.csv") 

After exploring in db beaver. We jouin on id patient id

In [3]:
QUERY = """
    SET SCHEMA 'diabetes';
    SELECT * FROM
            patient p
        LEFT JOIN
            blood_metrics bm
        ON p.id = bm.patientid
        LEFT JOIN
            pedigree_outcome po
        ON p.id = po.patientid
        LEFT JOIN
            skin s
        ON p.id = s.patientid
    ;
"""

Loads data from the database and stores it in pandas dataframes:

In [4]:
load_dotenv()

DB_CONFIG = {
    "database": os.getenv("DATABASE"),
    "user": os.getenv("USER_DB"),
    "password": os.getenv("PASSWORD"),
    "host": os.getenv("HOST"),
    "port": os.getenv("PORT")
}

DB_STRING = (
    "postgresql://{user}:{password}@{host}:{port}/{database}"
    .format(**DB_CONFIG)
)

In [5]:
db = create_engine(DB_STRING)

with db.connect() as conn:
    df = pd.read_sql(QUERY, conn)

Let us check that the data were retrieved.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1536 entries, 0 to 1535
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        1536 non-null   int64  
 1   Age                       1536 non-null   int64  
 2   pregnancies               1536 non-null   int64  
 3   bmi                       1536 non-null   float64
 4   insulin                   1536 non-null   int64  
 5   glucose                   1536 non-null   int64  
 6   bloodpressure             1536 non-null   int64  
 7   patientid                 1536 non-null   int64  
 8   id                        1536 non-null   int64  
 9   measurement_date          1536 non-null   object 
 10  diabetespedigreefunction  1536 non-null   float64
 11  outcome                   1536 non-null   int64  
 12  patientid                 1536 non-null   int64  
 13  id                        1536 non-null   int64  
 14  skinthic

In [7]:
df.sample(10)



,id,Age,pregnancies,bmi,insulin,glucose,bloodpressure,patientid,id,measurement_date,diabetespedigreefunction,outcome,patientid,id,skinthickness,patientid,id
1030,263,24,4,32.1,76,74,82,263,1031,2022-12-01,0.612,0,263,263,32,263,263
373,374,25,2,34.9,94,105,58,374,374,2022-12-13,0.225,0,374,374,40,374,374
62,63,36,5,25.0,0,44,62,63,63,2022-12-13,0.587,0,63,63,0,63,63
1338,571,39,3,32.5,83,34,102,571,1339,2022-12-01,0.270,0,571,571,0,571,571
170,171,36,6,30.8,0,102,82,171,171,2022-12-13,0.180,1,171,171,0,171,171
723,724,42,5,39.1,105,117,86,724,724,2022-12-13,0.251,0,724,724,30,724,724
1144,377,22,0,25.2,18,195,90,377,1145,2022-12-01,0.299,0,377,377,15,377,377
1101,334,44,12,23.6,1,176,99,334,1102,2022-12-01,0.137,0,334,334,0,334,334
278,279,57,5,24.9,0,114,74,279,279,2022-12-13,0.744,0,279,279,0,279,279
657,658,41,1,38.9,200,120,80,658,658,2022-12-13,1.162,0,658,658,48,658,658


Before storing, let us remove 

In [ ]:
df = df.loc[:, ~df.columns.duplicated()]

In [9]:
df.to_pickle(DF_PKL_PATH)
df.to_csv(DF_CSV_PATH, index=False)